# Installs

In [1]:
!pip install ftfy
!pip install catboost


[notice] A new release of pip is available: 24.0 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
import glob
import numpy as np
import re
from string import punctuation, whitespace
import warnings
warnings.filterwarnings("ignore")
import ftfy
from sklearn.utils import shuffle
import gc
import multiprocess as mp
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.base import BaseEstimator, TransformerMixin

# File Reading + Preps

Проблеми:
1. Багато неанглійських мов, що дуже ускладнює класифікацію.
    * Рішення: Видалити
2. Серед твітів, класифікованих як англійські, є німецькі та російські тексти.
    * Рішення: видалити через регекс

In [3]:
#../input/russian-troll-tweets/IRAhandle_tweets_1.csv
PATH = "input/russian-troll-tweets/"
filenames = glob.glob(os.path.join(PATH, "*.csv"))
full_ru_trolls = pd.concat((pd.read_csv(f) for f in filenames))
full_ru_trolls.drop(['external_author_id', 'region', 'harvested_date',
        'updates', 'account_type', 'new_june_2018', 'post_type',
        'account_category', 'following', 'followers', 'retweet'],
        axis=1, inplace=True)

full_ru_trolls = full_ru_trolls[full_ru_trolls.content.notnull()]
full_ru_trolls['troll'] = 1
full_ru_trolls_en = full_ru_trolls[full_ru_trolls.language == 'English'].copy(deep=True)

full_ru_trolls_en.rename(columns={'author': 'account', 'content': 'tweet'}, inplace=True)
full_ru_trolls_en = full_ru_trolls_en[~full_ru_trolls_en.tweet.str.contains('А-Яа-я')]

german_s = re.compile('(Ich )|(Sie )|(Ihnen )|( sich$)|( [Kk]?eine? )|( [Dd]as )|'+
           '^[Dd]as |^[Ss]ind | bist | und | sind | (?!(van|von|-)) der |' + 
           '[ ^][a-z]*ö|[ ^][a-z]*ä|[ ^][a-z]*ü')

full_ru_trolls_en = full_ru_trolls_en[~full_ru_trolls_en.tweet.str.contains(german_s)].copy(deep=True)

del full_ru_trolls
full_ru_trolls_en.drop(['language', 'publish_date'], axis=1, inplace=True)

Оскільки інфа не має чіткої структури, було прийнято рішення:
* Перейменувати всі стовпці імені облікового запису на "account", а всі стовпці з твітами - на "tweet"
* Не використовувати додаткові колонки які є різними для датасетів

In [4]:
sentiment140 = pd.read_csv('input/sentiment140/training.1600000.processed.noemoticon.csv',\
    encoding = 'Latin-1', names=('target', 'id', 'date', 'flag', 'username','tweet'))

sentiment140.drop(['target', 'id', 'date', 'flag'], axis=1, inplace=True)
sentiment140.rename(columns={'username': 'account'}, inplace=True)
sentiment140['troll'] = 0

PATH = "input/RawTwitterFeeds/"
filenames = glob.glob(os.path.join(PATH, "*.csv"))
celebs = pd.concat((pd.read_csv(f) for f in filenames))
celebs.drop(['Unnamed: 0', 'Unnamed: 0.1','id', 'date', 'link', 'retweet'], axis=1,inplace=True)

celebs.rename(columns={'author': 'account', 'text': 'tweet'}, inplace=True)
celebs = celebs[celebs.tweet.notnull()]
celebs['troll'] = 0

Об'єднання в стандартизований фрейм

In [5]:
used_cols = ['account', 'tweet', 'troll']
total_data = pd.concat([celebs[used_cols], sentiment140[used_cols], full_ru_trolls_en[used_cols]], ignore_index = True)

total_data

,account,tweet,troll
0,various,#DataScience Basics: #DataMining vs. #Statisti...,0
1,various,How to Become a #Data Scientist – Part 1: http...,0
2,various,@jesterxl @kdnuggets or just go with @tableau :),0
3,various,#Boston U. Online MS in Applied #Business #Ana...,0
4,various,#ICYMI Still Searching for ROI in #BigData Ana...,0
...,...,...,...
3901570,_YOUR_LIFESTYLE,Like me. Share me. Follow me http://t.co/VvhUO...,1
3901571,_YOUR_LIFESTYLE,.the main thing is to was love https://t.co/oq...,1
3901572,_YOUR_LIFESTYLE,Amazing! Found my script: Download ReBuild - W...,1
3901573,_YOUR_LIFESTYLE,Present Perfect http://t.co/vNu3i2Xh4M,1


In [6]:
def cleaning_and_counts(s):
    # Imports/sets here because it'd be executed in subroutine which executes independently from main code
    import ftfy, re, numpy as np
    from string import punctuation, whitespace
    
    # Всі коди дефісу(або аналогічних символів) які я знайшов в текстах
    dashes = [chr(int(d, 16)) for d in ['058A', '05BE', '1400', '1806', '2010', '2011',\
          '2012', '2013', '2014', '2015', '2053', '207B', '208B', '2212', '2E17', \
          '2E1A', '2E3A', '2E3B', '2E40', '2E5D', '301C', '3030', '30A0', 'FE31', \
          'FE32', 'FE58', 'FE63', 'FF0D', '10EAD']]
    dashes_compiled = re.compile('[' + ''.join(dashes) + ']+', flags = re.UNICODE)
    
    s = ftfy.fix_text(s)
    s = re.sub(dashes_compiled, '-', s)     # all dashes should be the same

    url_n = len(re.findall('https?://\\S+\\b', s)) # count urls
    s = re.sub('https?://\\S+\\b', '', s)   # and remove them

    hasht_n = len(re.findall(r'#\w+\b', s)) # count hashtags
    s = re.sub(r'#\w+\b', '', s)            # remove them

    handle_n = len(re.findall(r'@\w{1,15}\b', s)) # count handles
    s = re.sub(r'@\w{1,15}\b', '', s)       # remove them

    s = re.sub('pic\\.twitter\\.com/\\w+\\b', '', s)        # remove pictures. Not expected to impact overall results
    s = re.sub('\\s+', ' ', s)                              # reducing multiple whitespaces to one
    s = s.lstrip(whitespace + punctuation + '\xa0' + chr(8230))   # removing possible whitespaces in front
    s = s.rstrip(whitespace + '\xa0')         # and on the back
    l=''
    emoj_and_such = 0
    for ch in s:
        if ord(ch) < 8204:
            l += ch             # keep a symbol if not emoji or pictogram or such
        else:
            emoj_and_such += 1  # counting emojis and pictograms
    comma_n = len(re.findall(',', s))
    exl_n =  len(re.findall('!', s))
    dash_n = len(re.findall('-', s))
    a_an_n = len(re.findall(r'\b[Aa]n?\b', s))
    the_n = len(re.findall(r'\b[Tt]he\b', s))

    # reduce a number of repeated symbols to no more than 2 
    l = re.sub(r'(.)\1\1+', r'\1\1', l)
    length = len(l)

    words = [len(w) for w in re.findall(r'\b\w+\b', l)]
    if len(words)==0:
        average_word = 0
    else:
        average_word = np.max(words)
    
    return l, url_n, hasht_n, handle_n, emoj_and_such, exl_n, comma_n, dash_n, a_an_n, the_n, length, average_word


In [7]:
%%time

# Multithreaded pre-processing of dataframe
with mp.Pool(processes= mp.cpu_count()) as p:
    total_data['tuple'] = p.map(cleaning_and_counts, total_data.tweet)

CPU times: total: 15.5 s
Wall time: 1min 20s


Після обробки треба розпакувати результат в колонки

In [8]:
features = ("cleaned_tweet, url_n, hasht_n, handle_n, emoji_and_such, exl_n, comma_n, dash_n, a_an_n, the_n, length, average_word").split(', ')

for i in range(len(features)):
    if i == 0:
        total_data[features[i]] = total_data.tuple.apply(lambda t: t[i])
    else:
        total_data[features[i]] = total_data.tuple.apply(lambda t: t[i]).astype(np.uint8)

Очистка пам'яті, бо даних дуже багато

In [9]:
print(total_data.columns)
total_data.drop(['tuple'], axis=1,inplace=True)
del full_ru_trolls_en, celebs, sentiment140
gc.collect()

Index(['account', 'tweet', 'troll', 'tuple', 'cleaned_tweet', 'url_n',
       'hasht_n', 'handle_n', 'emoji_and_such', 'exl_n', 'comma_n', 'dash_n',
       'a_an_n', 'the_n', 'length', 'average_word'],
      dtype='object')


0

Видаляю акаунти, які містять менше 10 твітів.
* Причина: Аналіз діяльності облікових записів, в контексті якого 1 публікація не дає достатньо інформації.

In [10]:
min_count = 10
acc_properties = total_data[['account', 'troll']].groupby(['account'])\
    .agg(tweet_count=('account', 'size'),troll = ('troll','min'))\
    .reset_index()
kept_accs = acc_properties[acc_properties.tweet_count >= min_count]
restricted = total_data[total_data.account.isin(kept_accs.account)].copy(deep=True)
del total_data

num_cols = features[1:]
restricted.drop(['tweet', 'cleaned_tweet'], axis=1, inplace=True)
restricted

,account,troll,url_n,hasht_n,handle_n,emoji_and_such,exl_n,comma_n,dash_n,a_an_n,the_n,length,average_word
0,various,0,1,4,1,0,0,0,0,0,0,15,6
1,various,0,1,1,1,0,0,0,1,1,0,39,9
2,various,0,0,0,3,0,0,0,0,0,0,18,4
3,various,0,1,4,1,0,0,1,1,0,0,57,10
4,various,0,1,2,0,0,1,0,0,0,0,55,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3901564,_SOLOMONALBERT_,1,0,2,0,0,0,0,0,2,0,81,7
3901565,_SOLOMONALBERT_,1,2,4,1,0,0,0,0,0,0,12,7
3901566,_SOLOMONALBERT_,1,0,2,0,1,0,0,0,0,0,64,10
3901567,_SOLOMONALBERT_,1,0,3,0,0,0,0,0,0,1,52,9


Обчислення персентилів і шафл даних

In [11]:
def percentile_calc(data, groupby_col, num_cols, percentile_list):
    non_numeric = [col_name for col_name in data.columns if col_name not in num_cols]
    for qu in percentile_list: 
        percentiles = data.groupby(groupby_col).quantile(q=qu/100).reset_index()
        cols_to_change = {col : col +'_' + str(qu) for col in num_cols}
        percentiles.rename(columns=cols_to_change, inplace=True)
        if qu == percentile_list[0]:
            all_percentiles = percentiles
        else:
            all_percentiles = pd.merge(all_percentiles, percentiles, how = "left",\
                                       on = non_numeric)
    return all_percentiles

In [12]:
%%time
all_percentiles = percentile_calc(restricted[['account', 'troll']+num_cols], \
                                 groupby_col='account', num_cols=num_cols,
                                 percentile_list=range(10, 100, 10))
    
new_features = all_percentiles.columns[2:]
all_percentiles = shuffle(all_percentiles).reset_index(drop = True)
all_percentiles

CPU times: total: 6.77 s
Wall time: 9.3 s


,account,troll,url_n_10,hasht_n_10,handle_n_10,emoji_and_such_10,exl_n_10,comma_n_10,dash_n_10,a_an_n_10,...,hasht_n_90,handle_n_90,emoji_and_such_90,exl_n_90,comma_n_90,dash_n_90,a_an_n_90,the_n_90,length_90,average_word_90
0,dannyatticus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.4,1.0,0.0,2.0,1.0,119.8,9.4
1,mikeinsd77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,125.8,9.0
2,chricket,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.7,116.5,9.7
3,victoriaamarie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.9,0.0,1.0,0.0,0.0,1.0,1.0,118.7,9.0
4,MRMUSTACHEE,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.4,0.0,1.0,0.0,0.4,1.0,78.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23267,tinknevertalks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.4,0.0,1.0,1.4,120.6,10.0
23268,ashaazaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.3,0.0,1.0,0.3,70.6,9.0
23269,BADDESTNLA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,86.8,9.0
23270,LauraKalbag,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.2,1.0,0.2,1.0,1.0,114.6,9.0


# Модель. Тренування і тести

Кастомний scikit-learn Transformer для виключення фіч методами MI та Кореляції Пірсона (протестовано і гарно працює). 

In [13]:
def _drop_correlated(data, score_ordered_cols, max_corr, method='pearson'):
    new = [[score_ordered_cols[0]], [0]]
    corr_matrix = data[score_ordered_cols].corr(method).values
    N = len(score_ordered_cols)
    for i in range(1, N):
        tr = corr_matrix[new[1], i]
        if sum(np.abs(tr) > max_corr) == 0:
            new[0] += [score_ordered_cols[i]]
            new[1] += [i]
    return new[0]


class feature_reduction(BaseEstimator, TransformerMixin):
    def __init__(self, min_mi=.001, max_corr=.7, n_neighbors=11):
        self.min_mi = min_mi
        self.max_corr = max_corr
        self.n_neighbors = n_neighbors

    def fit(self, X:pd.DataFrame, y):
        X = X.copy(deep=True)
        columns = X.columns
        mi = mutual_info_classif(X.values, y, n_neighbors= self.n_neighbors)
        cols_mi = list(zip(columns, mi))
        cols_mi.sort(reverse=True, key=lambda x: x[1])
        cols_mi = [pair[0] for pair in cols_mi if pair[1] > self.min_mi]
        new_cols = _drop_correlated(X[cols_mi], cols_mi, max_corr=self.max_corr)
        self.selected_cols = new_cols
        return self

    def transform(self, X, y=None):
        return X[self.selected_cols]

In [14]:
def searchcv_best(estimator):
    print(" The best AUC score on a train set is:\n", estimator.best_score_)
    print(" The best parameters are:\n", estimator.best_params_)

def scores(y_test, y_pred):
    print('===== Metrics on a test set =====')
    print("A confusion matrix is:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print('accuracy:', accuracy_score(y_test, y_pred), 
        '\nf1:', f1_score(y_test, y_pred),
        '\nprecision:', precision_score(y_test, y_pred), 
        '\nrecall:', recall_score(y_test, y_pred), 
        '\nroc_auc:', roc_auc_score(y_test, y_pred))

In [15]:
def train_pipe_cv(data, columns, parameters, fr_params, cbc_params,cv=5, train_size=.8, scoring='roc_auc'):
    X_train, X_test, y_train, y_test = train_test_split(data[columns], data['troll'], train_size = train_size, stratify = data['troll'])
    fr = feature_reduction(**fr_params)
    model = CatBoostClassifier(**cbc_params)
    pipe = Pipeline([('mi_calc', fr), ('catboost', model)])
    search = GridSearchCV(pipe, param_grid=parameters, cv = cv, scoring= scoring, n_jobs=-1, verbose=1)
    search.fit(X_train, y_train)
    searchcv_best(search)
    y_pred = search.predict(X_test)
    scores(y_test, y_pred)
    return search, search.best_params_

In [16]:
%%time
pipe_params = {'mi_calc__max_corr': [0.7, 0.75],
               'mi_calc__n_neighbors': [5, 19],
               'catboost__depth': [9, 11]}

fr_params = {'min_mi': 0.0005}
cbc_params = {
    'loss_function': 'Logloss', 
    
    'rsm': 0.25, 
    'l2_leaf_reg': 4, 
    'custom_metric': 'AUC',
    
    'logging_level': 'Silent',
    #'task_type': 'GPU'
}

model, cv_best_params = train_pipe_cv(all_percentiles, new_features,parameters=pipe_params, fr_params=fr_params,\
                 cbc_params=cbc_params, scoring ='roc_auc')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
 The best AUC score on a train set is:
 0.9994725742386127
 The best parameters are:
 {'catboost__depth': 11, 'mi_calc__max_corr': 0.7, 'mi_calc__n_neighbors': 19}
===== Metrics on a test set =====
A confusion matrix is:
[[4368   10]
 [   6  271]]
accuracy: 0.99656283566058 
f1: 0.9713261648745519 
precision: 0.9644128113879004 
recall: 0.9783393501805054 
roc_auc: 0.9880276010838572
CPU times: total: 17.1 s
Wall time: 5min 49s


In [17]:
for key in cv_best_params:
    if key[:3] == 'cat':
        cbc_params[key.split('__')[1]] = cv_best_params[key]
    else:
        fr_params[key.split('__')[1]] = cv_best_params[key]

fr_cols = feature_reduction(**fr_params)
fr_cols.fit(all_percentiles[new_features], all_percentiles.troll)
new_cols = fr_cols.selected_cols
new_cols

['hasht_n_90',
 'url_n_90',
 'url_n_50',
 'hasht_n_50',
 'handle_n_70',
 'average_word_90',
 'average_word_60',
 'handle_n_40',
 'emoji_and_such_90',
 'handle_n_90',
 'hasht_n_10',
 'length_30',
 'average_word_20',
 'exl_n_90',
 'dash_n_90',
 'length_80',
 'comma_n_90',
 'handle_n_20',
 'comma_n_70',
 'exl_n_50',
 'a_an_n_70',
 'a_an_n_80',
 'the_n_90',
 'comma_n_50',
 'emoji_and_such_70',
 'the_n_60',
 'a_an_n_60',
 'a_an_n_90',
 'the_n_80',
 'the_n_70',
 'a_an_n_50',
 'exl_n_20',
 'the_n_50',
 'dash_n_70',
 'emoji_and_such_50',
 'comma_n_30',
 'a_an_n_30']

Список фіч, упорядкованих за обчисленою важливістю. 

Вони дещо змінюються з кожним повторним запуском, хоча перші, як правило, залишаються вище в списку.

In [18]:
cbc_best = CatBoostClassifier(**cbc_params)
cbc_best.fit(all_percentiles[new_cols], all_percentiles.troll.values)
var_importance = list(zip(cbc_best.feature_importances_, new_cols))
var_importance.sort(key=lambda x:x[0], reverse=True)
print("A number of variables is "+str(len(new_cols)))
var_importance

A number of variables is 37


[(8.857295146550172, 'url_n_90'),
 (8.713204965596162, 'average_word_90'),
 (6.997567220013056, 'hasht_n_90'),
 (6.991269212699155, 'length_30'),
 (5.757704940891846, 'exl_n_90'),
 (5.489514988849892, 'length_80'),
 (5.4434630881202635, 'handle_n_70'),
 (5.311304124567704, 'comma_n_90'),
 (4.952976728744958, 'handle_n_40'),
 (4.7492990160008555, 'handle_n_20'),
 (3.9950592147743453, 'average_word_20'),
 (3.2938309130147645, 'handle_n_90'),
 (3.21619381679535, 'average_word_60'),
 (3.013813751932447, 'dash_n_90'),
 (2.740493567523952, 'comma_n_70'),
 (2.4670590146885587, 'the_n_90'),
 (2.235395486866762, 'exl_n_50'),
 (2.0266133405015645, 'a_an_n_90'),
 (2.0062230128522502, 'a_an_n_70'),
 (1.6342957518359635, 'url_n_50'),
 (1.4573030401368232, 'the_n_80'),
 (1.126306837164903, 'a_an_n_80'),
 (1.0748099970969158, 'hasht_n_50'),
 (0.9635185636293575, 'the_n_70'),
 (0.9607086633100631, 'comma_n_50'),
 (0.912655657107789, 'emoji_and_such_90'),
 (0.876549168701829, 'dash_n_70'),
 (0.84076439

# Висновки 
Як ми бачимо, найважливішим для виявлення тролів є те, як вони дотримуються своїх інструкцій. 
- Боти повинні включати конкретні слова у свої тексти, тому середня довжина слова відрізняється від того, що використовують звичайні користувачі. 
- Боти використовують кілька способів збільшити свою аудиторію: посилання, хештеги, хендлери Twitter, і це дуже помітно. 
- Боти швидше за все мають вказівки щодо довжини публікації. Це дуже зручно, тому що потенційно є можливість налаштувати фільтри для виявлення найбільш значущих показників, а вже потім перевірити всю активність облікового запису.
- ! Виявилося, що запропоновані фічі не дуже залежать від мов, а здебільшого від активності акаунта троля. Таким чином, ми можемо застосувати цю роботу для інших мов, а не обмежуватися російськими тролями, які розміщують англійські тексти.

In [23]:
#model.save_model("catboost_model.cbm", format="cbm")
temp = model.best_estimator_.named_steps['catboost']
temp.save_model("catboost_model.cbm", format="cbm")

import joblib
joblib.dump(model, 'model_test.pkl')

['model_test.pkl']

# ! Realtime

In [ ]:
# msg = """Dan Bongino: ""Nobody trolls liberals better than Donald Trump."" Exactly!  https://t.co/AigV93aC8J #asd #sdhfj #asd #asd #asd """
# test_tweet = [{"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               {"account": "Me My Mus", "tweet": msg},
#               ]
#test_df = pd.DataFrame(test_tweet)

test_df = pd.read_csv("test.csv")
test_df = test_df[['account', 'tweet']]
#test_df.to_csv("test_data.csv", index=False)
test_df.info()

# Cleaning
with mp.Pool(processes= mp.cpu_count()) as p:
    test_df['tuple'] = p.map(cleaning_and_counts, test_df.tweet)

# Memory Optimization
features = ("cleaned_tweet, url_n, hasht_n, handle_n, emoji_and_such, exl_n, comma_n, dash_n, a_an_n, the_n, length, average_word").split(', ')

for i in range(len(features)):
    if i ==0:
        test_df[features[i]] = test_df.tuple.apply(lambda t: t[i])
    else:
        test_df[features[i]] = test_df.tuple.apply(lambda t: t[i]).astype(np.uint8)

print(test_df.columns)
test_df.drop(['tuple'], axis=1,inplace=True)
gc.collect()

print(test_df)

# Check if total account tweets > 10

_min_count = 10
_acc_properties = test_df[['account']].groupby(['account'])\
    .agg(tweet_count=('account', 'size'))\
    .reset_index()

print(_acc_properties)
_kept_accs = _acc_properties[_acc_properties.tweet_count >= _min_count]
_restricted = test_df[test_df.account.isin(_kept_accs.account)].copy(deep=True)
#del total_data

_num_cols = features[1:]
_restricted.drop(['tweet', 'cleaned_tweet'], axis=1, inplace=True)
print("Restricted")
_restricted


# Generate features
_all_percentiles = percentile_calc(_restricted[['account']+num_cols], \
                                 groupby_col='account', num_cols=_num_cols,
                                 percentile_list=range(10, 100, 10))
    
_new_features = _all_percentiles.columns[2:]
print(_all_percentiles)

preditction = model.predict(_all_percentiles)
prediction_proba = model.predict_proba(_all_percentiles)[0]

print(f"Proba: {prediction_proba}")


print("\nPrediction: " + ("BOT Detected" if preditction == 1 else "Regular User"))
print("Troll Account Probability: " + str(prediction_proba[1]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   account  108 non-null    object
 1   tweet    108 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB
Index(['account', 'tweet', 'tuple', 'cleaned_tweet', 'url_n', 'hasht_n',
       'handle_n', 'emoji_and_such', 'exl_n', 'comma_n', 'dash_n', 'a_an_n',
       'the_n', 'length', 'average_word'],
      dtype='object')
    account                                              tweet  \
0    10_GOP  "We have a sitting Democrat US Senator on tria...   
1    10_GOP  Marshawn Lynch arrives to game in anti-Trump s...   
2    10_GOP  Daughter of fallen Navy Sailor delivers powerf...   
3    10_GOP  JUST IN: President Trump dedicates Presidents ...   
4    10_GOP  19,000 RESPECTING our National Anthem! #StandF...   
..      ...                                                ...   
103  10_GOP  Trump after milita